In [3]:
!pip3 install darkskylib
!pip3 install vayu

     |████████████████████████████████| 931kB 6.3MB/s 
     |████████████████████████████████| 10.9MB 41.8MB/s 
     |████████████████████████████████| 14.7MB 314kB/s 


In [0]:
import pandas as pd
import numpy as np
from darksky import forecast
import datetime as dt

## Calander Plot Function

In [0]:
from vayu import calendarPlot 

## With Open AQ Data

Downlaod data for IGI Airport T3, Delhi, India for a paricular year (using this only last 90 days data can fetched).
See https://openaq.org/#/location/IGI%20Airport%20(T3)%2C%20Delhi%20-%20IMD?_k=y096un

In [0]:
start_date = "2020-03-15"
end_date = "2020-05-08"
pollutant = "pm25"
url = "https://api.openaq.org/v1/measurements?format=csv&location=IGI%20Airport%20%28T3%29%2C%20Delhi%20-%20IMD&date_from="+start_date+"T18%3A30%3A00.000Z&date_to="+end_date+"T18%3A30%3A00.000Z&parameter%5B0%5D=pm25&parameter%5B1%5D=pm10&parameter%5B2%5D=o3&parameter%5B3%5D=no2&parameter%5B4%5D=co&parameter%5B5%5D=bc&parameter%5B6%5D=so2"

In [7]:
!wget -O pollutants.csv "{url}"

--2020-05-08 14:37:19--  https://api.openaq.org/v1/measurements?format=csv&location=IGI%20Airport%20%28T3%29%2C%20Delhi%20-%20IMD&date_from=2020-03-15T18%3A30%3A00.000Z&date_to=2020-05-08T18%3A30%3A00.000Z&parameter%5B0%5D=pm25&parameter%5B1%5D=pm10&parameter%5B2%5D=o3&parameter%5B3%5D=no2&parameter%5B4%5D=co&parameter%5B5%5D=bc&parameter%5B6%5D=so2
Resolving api.openaq.org (api.openaq.org)... 13.35.112.47, 13.35.112.73, 13.35.112.53, ...
Connecting to api.openaq.org (api.openaq.org)|13.35.112.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3297873 (3.1M) [text/csv]
Saving to: ‘pollutants.csv’

pollutants.csv      100%[===================>]   3.14M  2.95MB/s    in 1.1s    

2020-05-08 14:37:22 (2.95 MB/s) - ‘pollutants.csv’ saved [3297873/3297873]



Change the date range for the year

In [8]:
df = pd.read_csv('pollutants.csv')
df = df[df.parameter== pollutant]
df = df.rename(columns={"value": pollutant})

df['date']= pd.to_datetime(df['local'])
df = df.groupby([df['date'].dt.date]).mean()

### change the range ###
idx = pd.date_range('2019-01-01' , '2019-12-31')

df = df.reindex(idx, fill_value=np.nan)
df = df.reset_index()
df = df.rename(columns={"index": "date"})
df

,date,pm25,latitude,longitude
0,2019-01-01,NaN,NaN,NaN
1,2019-01-02,NaN,NaN,NaN
2,2019-01-03,NaN,NaN,NaN
3,2019-01-04,NaN,NaN,NaN
4,2019-01-05,NaN,NaN,NaN
...,...,...,...,...
360,2019-12-27,NaN,NaN,NaN
361,2019-12-28,NaN,NaN,NaN
362,2019-12-29,NaN,NaN,NaN
363,2019-12-30,NaN,NaN,NaN


## With Berkley Earth data

In [9]:
!wget http://berkeleyearth.lbl.gov/air-quality/maps/cities/India/Delhi/Delhi.txt
df = pd.read_csv("Delhi.txt", delimiter='\t', comment="%",header = None)
df.columns=[
    "Year", "Month", "Day", "UTC Hour", 
    "pm25", "pm10", "Retrospective"
]
df['date'] = pd.to_datetime(df[['Year','Month','Day']])
df = df.groupby([df['date'].dt.date]).mean()

### change the date range ###
idx = pd.date_range('2019-01-01' , '2019-12-31')

df = df.reindex(idx, fill_value=np.nan)
df = df.reset_index()
df = df.rename(columns={"index": "date"})
df

--2020-05-08 14:37:24--  http://berkeleyearth.lbl.gov/air-quality/maps/cities/India/Delhi/Delhi.txt
Resolving berkeleyearth.lbl.gov (berkeleyearth.lbl.gov)... 128.3.29.26
Connecting to berkeleyearth.lbl.gov (berkeleyearth.lbl.gov)|128.3.29.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 821293 (802K) [text/plain]
Saving to: ‘Delhi.txt’

Delhi.txt           100%[===================>] 802.04K   613KB/s    in 1.3s    

2020-05-08 14:37:26 (613 KB/s) - ‘Delhi.txt’ saved [821293/821293]



,date,Year,Month,Day,UTC Hour,pm25,pm10,Retrospective
0,2019-01-01,2019.0,1.0,1.0,11.500000,241.550833,0.0,0.0
1,2019-01-02,2019.0,1.0,2.0,11.500000,303.945833,0.0,0.0
2,2019-01-03,2019.0,1.0,3.0,11.500000,278.686667,0.0,0.0
3,2019-01-04,2019.0,1.0,4.0,11.565217,240.369130,0.0,0.0
4,2019-01-05,2019.0,1.0,5.0,11.500000,220.882500,0.0,0.0
...,...,...,...,...,...,...,...,...
360,2019-12-27,2019.0,12.0,27.0,8.000000,201.344118,0.0,0.0
361,2019-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
362,2019-12-29,2019.0,12.0,29.0,19.500000,320.176250,0.0,0.0
363,2019-12-30,2019.0,12.0,30.0,11.500000,243.880833,0.0,0.0


##Adding wind speed and wind direction to the data frame

Go to https://darksky.net/dev and generate your API key to fetch wind data.

In [0]:
API_key = 'blahblahblahblah'
latitude = 28.554462433
longitude = 77.087898254
location = API_key, latitude, longitude  

In [14]:
### change the date ###
date1 = dt.datetime(2019, 1, 1)
date2 = dt.datetime(2019, 12, 31)

def daterange(d1, d2):
  return (d1 + dt.timedelta(days=i) for i in range((d2 - d1).days + 1))

for ((index,row), d) in zip(df.iterrows(), daterange(date1, date2)):
  data = forecast(*location, time = d.isoformat())
  df.loc[index, 'wd'] = data['daily']['data'][0]['windBearing']
  df.loc[index, 'ws'] = data['daily']['data'][0]['windSpeed']

df.head()

HTTPError: ignored

In [0]:
calendarPlot(df, 'pm25','2020','India')